<a href="https://colab.research.google.com/github/dunkelweizen/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/Copy_of_LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

import urllib3

In [0]:
max_features = 20000
batch_size = 32

In [0]:
data = open('/content/complete_works_shakespeare', encoding='utf-8').read()


In [4]:
len(data)

5573152

In [0]:
import re

regex = re.compile(r'[\n\r\t]')
data = regex.sub(' ', data)

In [0]:
#data = data[1000000:2000000]

In [0]:
chars = list(set(data))

In [0]:
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [9]:
# Create the sequence data
maxlen = 40
step = 5

encoded = [char_int[c] for c in data]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1114623


In [0]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [11]:
x.shape

(1114623, 40, 105)

In [12]:
y.shape

(1114623, 105)

In [13]:
np.count_nonzero(x)

44584920

In [0]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(data) - maxlen - 1)
    
    generated = ''
    
    # sentence = text[start_index: start_index + maxlen]
    sentence = "I wish to see the color of your eyes and"
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
model.fit(x, y,
          batch_size=100,
          epochs=200,
          callbacks=[print_callback])

Epoch 1/200
11147/11147 [==============================] - ETA: 0s - loss: 1.8939 - accuracy: 0.4590
----- Generating text after Epoch: 0
----- Generating with seed: "I wish to see the color of your eyes and"
I wish to see the color of your eyes and his a but of her wespidice,     Most ane thoy Endias, bedrout not lent you leadly wos where hard desco of Vissige on ander) How mist?   RoMight Lechs bent Tentin, it do sin; not be in plaige knodl; as it that.  SCONERIA. Hou by marks, art With thoue fall blend good the frigs. Wheriogioncled Puttro; a majes with frimndle now,     But so will would be goud on such a treased withis, lane art mid! hi
11147/11147 [==============================] - 76s 7ms/step - loss: 1.8939 - accuracy: 0.4590
Epoch 2/200
11140/11147 [============================>.] - ETA: 0s - loss: 1.7080 - accuracy: 0.5062
----- Generating text after Epoch: 1
----- Generating with seed: "I wish to see the color of your eyes and"
I wish to see the color of your eyes and mared.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


my heart, engle other freal that Caesar in all th' most stopper thy power might     Strange a pipering with the line true to do you appear, And operborn
11147/11147 [==============================] - 76s 7ms/step - loss: 1.3698 - accuracy: 0.5921
Epoch 24/200
11144/11147 [============================>.] - ETA: 0s - loss: 1.3669 - accuracy: 0.5930
----- Generating text after Epoch: 23
----- Generating with seed: "I wish to see the color of your eyes and"
I wish to see the color of your eyes and     bost cry of his Broughter See will friendling.   DERDYMAL. Speak the losess, Gower hark their greatess.  THERSITES. A tears so, by my lord; and, rest on     Did sun at him! Enter Arcap'd: Phoise BUCKINGHANNE OF OLEX. Atto Pates Forth, while he were sail in engaged By Haste-sempts of the hase.   OLLA. I pray you by Edmund Lastunat’s will the Endia off and with sparrow.     I am: feels into my 
11147/11147 [==============================] - 76s 7ms/step - loss: 1.3670 - accuracy: 0.5930
Epoch 2

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN